In [1]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch.nn as nn
import torch.optim as optim
from model import UNET
from utils import (
    load_checkpoint,
    save_checkpoint,
    get_loaders_masks,
    get_loaders_landmarks,
    check_accuracy,
    save_predictions_as_imgs,
    train_fn)

In [2]:
# Hyperparameters
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 64
NUM_EPOCHS =3
NUM_WORKERS = 2
IMAGE_HEIGHT = 112
IMAGE_WIDTH = 112
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = "../data/train_images/"
TRAIN_MASK_DIR = "../data/train_masks/"
VAL_IMG_DIR = "../data/val_images/"
VAL_MASK_DIR = "../data/val_masks/"

#Transformations
train_transform = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ]
)

val_transforms = A.Compose(
    [
        A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ]
)

#Model
model = UNET(in_channels=3, out_channels=8).to(DEVICE)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

#Batches
train_loader, val_loader = get_loaders_landmarks(
    TRAIN_IMG_DIR,
    TRAIN_MASK_DIR,
    VAL_IMG_DIR,
    VAL_MASK_DIR,
    BATCH_SIZE,
    train_transform,
    val_transforms,
    NUM_WORKERS,
    PIN_MEMORY,
)

model_type="landmarks"

In [3]:

def main():

    if LOAD_MODEL:
        load_checkpoint(torch.load("my_checkpoint.pth.tar"), model)

    scaler = torch.cuda.amp.GradScaler()

    for epoch in range(NUM_EPOCHS):
        train_fn(train_loader, model, optimizer, loss_fn, scaler, DEVICE,model_type)

        # save model
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        #save_checkpoint(checkpoint)

        #check accuracy
        check_accuracy(val_loader, model, device=DEVICE, model_type=model_type)

        path = f"../saved_images/epoch_{epoch}"

        # Save predictions
        save_predictions_as_imgs(
            val_loader, model, folder=path, device=DEVICE, model_type=model_type
        )


if __name__ == "__main__":
    main()

100%|██████████| 16/16 [00:17<00:00,  1.11s/it, loss=0.642]


Got 9801061/12845056 with acc 76.30
Dice score: 76.30


100%|██████████| 16/16 [00:17<00:00,  1.10s/it, loss=0.581]


Got 9875653/12845056 with acc 76.88
Dice score: 76.88


100%|██████████| 16/16 [00:17<00:00,  1.10s/it, loss=0.545]


Got 10388974/12845056 with acc 80.88
Dice score: 80.88


In [4]:
imagen, heatmap, mascara = next(iter(train_loader))

In [5]:
heatmap.shape

torch.Size([64, 8, 112, 112])

In [11]:
def unravel_index(indices,shape):
    shape = torch.tensor(shape)
    indices = indices % shape.prod()  # prevent out-of-bounds indices

    coord = torch.zeros(indices.size() + shape.size(), dtype=int)

    for i, dim in enumerate(reversed(shape)):
        coord[..., i] = indices % dim
        indices = indices // dim

    return coord.flip(-1)

In [18]:
t.shape

torch.Size([8, 112, 112])

In [20]:
t[0].argmax()

tensor(4525)

In [21]:
coor=[]
t =heatmap[0]
for i in range(t.shape[0]):
    coordenada = unravel_index(t[i].argmax(), t.shape[1:])
    coor.append(coordenada)

points =np.array(coor)        

In [34]:
import cv2

In [39]:
img = np.zeros((112,112))
print(points.shape)
for punto in points:
    print(punto)
    cv2.circle(img, punto, 10, (0, 255, 0), -1)

img_rsz = cv2.resize(img, (500, 500))
cv2.imshow('Image', img_rsz)
cv2.waitKey(0)
cv2.destroyAllWindows()

(8, 2)
[40 45]
[42 46]
[50 51]
[59 59]
[71 67]
[49 50]
[59 56]
[67 61]
